In [ ]:
import numpy as np
import os
from scipy.linalg import eigh, qr, null_space
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'DejaVu Sans'
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from scipy.sparse import kron, identity, csr_matrix, csc_matrix, lil_matrix, dok_matrix, issparse, coo_matrix
from scipy.sparse.linalg import eigsh, eigs, LinearOperator, ArpackNoConvergence
from scipy.optimize import curve_fit
from qutip import Qobj, ptrace, entropy_vn, qeye, tensor
from tqdm import tqdm   
import time
from itertools import product
from functools import reduce
import torch
import torch.optim as optim
from torch.autograd import Variable
import sympy as sp
from collections import Counter

In [ ]:
os.environ["ARPACK_VERBOSE"] = "1"

In [ ]:
def pauli_x():
    """Pauli X matrix."""
    return np.array([[0, 1], [1, 0]])

def pauli_z():
    """Pauli Z matrix."""
    return np.array([[1, 0], [0, -1]])

def dodecahedral_bonds(): #20 vertices
    """
    Defines the connectivity of a true 20-vertex dodecahedral molecular structure.

    Returns:
        list of tuples: Each tuple (i, j) represents a bond between spin i and spin j.
    """
    bonds = [
    (0, 13), (0, 14), (0, 15),
    (1, 4), (1, 5), (1, 12),
    (2, 6), (2, 13), (2, 18),
    (3, 7), (3, 14), (3, 19),
    (4, 10), (4, 18),
    (5, 11), (5, 19),
    (6, 10), (6, 15),
    (7, 11), (7, 15),
    (8, 9), (8, 13), (8, 16),
    (9, 14), (9, 17),
    (10, 11),
    (12, 16), (12, 17),
    (16, 18),
    (17, 19)
]

    return bonds


def transverse_field_ising_dodecahedral(N, J, h):
    """
    Constructs the Hamiltonian for the transverse field Ising model on a dodecahedral molecular structure.

    Parameters:
        N (int): Number of spins (should match the dodecahedral molecule, typically N=12).
        J (float): Interaction strength.
        h (float): Transverse field strength.
    
    Returns:
        H (scipy.sparse.csr_matrix): The Hamiltonian matrix in sparse format.
    """
    if N != 20:
        raise ValueError("Dodecahedral molecules typically have N = 20 sites.")

    # Sparse identity matrix
    I = identity(2, format="csr")
    
    # Pauli matrices as sparse matrices
    X = csr_matrix(pauli_x())
    Z = csr_matrix(pauli_z())
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**N, 2**N), dtype=np.float64)

    # Get dodecahedral bonds
    bonds = dodecahedral_bonds()

    # Interaction term: J * sigma_i^x * sigma_j^x for dodecahedral connectivity
    for i, j in bonds:
        term = 1
        for k in range(N):
            if k == i or k == j:
                term = kron(term, X, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += J * term
    
    # Transverse field term: -h * sigma_i^z
    for i in range(N):
        term = 1
        for j in range(N):
            if j == i:
                term = kron(term, Z, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += -h * term
    
    return H

def ising_dodecahedron(N, J):
    """
    Constructs the Hamiltonian for the transverse field Ising model on a dodecahedral molecular structure without transverse field.

    Parameters:
        N (int): Number of spins (should match the dodecahedral molecule, typically N=20).
        J (float): Interaction strength.
    
    Returns:
        H (scipy.sparse.csr_matrix): The Hamiltonian matrix in sparse format.
    """
    if N != 20:
        raise ValueError("Dodecahedral molecules typically have N = 20 sites.")

    # Sparse identity matrix
    I = identity(2, format="csr")
    
    # Pauli matrices as sparse matrices
    X = csr_matrix(pauli_x())
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**N, 2**N), dtype=np.float64)

    # Get Dodecahedron bonds
    bonds = dodecahedral_bonds()

    # Interaction term: J * sigma_i^x * sigma_j^x for Dodecahedron connectivity
    for i, j in bonds:
        term = 1
        for k in range(N):
            if k == i or k == j:
                term = kron(term, X, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += J * term
    
    return H

def transverse_field_dodecahedral(N, h):
    """
    Constructs the Hamiltonian for the transverse field Ising model on a dodecahedral molecular structure.

    Parameters:
        N (int): Number of spins (should match the icosahedral molecule, typically N=20).
        J (float): Interaction strength.
        h (float): Transverse field strength.
    
    Returns:
        H (scipy.sparse.csr_matrix): The Hamiltonian matrix in sparse format.
    """
    if N != 20:
        raise ValueError("Dodecahedral molecules typically have N = 20 sites.")

    # Sparse identity matrix
    I = identity(2, format="csr")
    
    # Pauli matrices as sparse matrices
    Z = csr_matrix(pauli_z())
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**N, 2**N), dtype=np.float64)

    # Get dodecahedral bonds
    bonds = dodecahedral_bonds()

    # Transverse field term: -h * sigma_i^z
    for i in range(N):
        term = 1
        for j in range(N):
            if j == i:
                term = kron(term, Z, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += -h * term
    
    return H

def H_shifted(H0, x):
    H_s = H0 - x * identity(H0.shape[0], format="csr")
    H_s = H_s.tocsr()
    return H_s


def H_shifted_sq(H0, x):
    H_s = H0 - x * identity(H0.shape[0], format="csr")
    H_s_sq = H_s @ H_s
    H_s_sq = H_s_sq.tocsr()
    return H_s_sq

In [ ]:
N = 20  # Number of spins
J = 1.0  # Interaction strength
h = 3.0  # Transverse field strength # this is the value in the paper. maybe try other values too, including the critical value one (h=J=1)


# Assuming transverse_field_ising is defined and returns a sparse Hermitian matrix
H = transverse_field_ising_dodecahedral(N, J, h)
Hi = ising_dodecahedron(N, J)
Htf = transverse_field_dodecahedral(N, h)

In [ ]:
eivt, eigt = eigsh(H, k=10, sigma= -18.0, which='LM')
#print(f"energy eigenvalue: {eiv}")
print(f"total energy eigenvalues: {eivt}")

In [ ]:
# Check if the eigenvectors from Htf are also eigenvectors of Hi and analyze sparsity
print("Analyzing eigenvectors from Htf (sigma=-18.1) for Hi eigenvalue properties and sparsity...")

# Tolerance for eigenvalue and sparsity analysis
eigenvalue_tol = 1e-10
sparsity_tol = 1e-8
total_components = 2**20  # 2^20 for 20 qubits

print(f"Total components in each eigenvector: {total_components}")
print(f"Using sparsity threshold: {sparsity_tol}")
print(f"Using eigenvalue consistency threshold: {eigenvalue_tol}")
print("-" * 80)

hi_eigenvalues = []
is_hi_eigenvector = []
sparsity_counts = []
zero_eigenvalue_count = 0

for i in range(len(eiv)):
    eigvec = eig[:, i]
    htf_eigenval = eiv[i]
    
    print(f"\nEigenvector {i}: Htf eigenvalue = {htf_eigenval:.8f}")
    
    # Test if it's an eigenvector of Hi
    Hi_result = Hi @ eigvec
    
    # Find non-zero components to check eigenvalue consistency
    nonzero_mask = np.abs(eigvec) > eigenvalue_tol
    nonzero_count = np.sum(nonzero_mask)
    
    if nonzero_count > 0:

        # Calculate ratios Hi*vec / vec for non-zero components
        ratios = Hi_result[nonzero_mask] / eigvec[nonzero_mask]
        ratio_std = np.std(ratios)
        ratio_mean = np.mean(ratios)
        
        # Check if ratios are consistent (indicating eigenvector property)
        is_eigenvector_i = ratio_std < eigenvalue_tol        
        
        if is_eigenvector_i:
            hi_eigenval = ratio_mean
            hi_eigenvalues.append(hi_eigenval)
            is_hi_eigenvector.append(True)

            # Check if eigenvalue is effectively zero
            is_zero_eigenval = abs(hi_eigenval) < eigenvalue_tol
            if is_zero_eigenval:
                zero_eigenvalue_count += 1
            
            print(f"  ✓ IS eigenvector of Hi")
            print(f"  Hi eigenvalue = {hi_eigenval:.10f}")
            print(f"  Is Hi eigenvalue zero? {is_zero_eigenval}")
            print(f"  Ratio std deviation = {ratio_std:.2e}")
        else:
            hi_eigenvalues.append(None)
            is_hi_eigenvector.append(False)
            print(f"  ✗ NOT an eigenvector of Hi")
            print(f"  Ratio std deviation = {ratio_std:.2e} (too large)")
            print(f"  Sample ratios: {ratios[:5]}")

    else:
        hi_eigenvalues.append(None)
        is_hi_eigenvector.append(False)
        print(f"  ✗ No significant non-zero components")        
    
    # Analyze sparsity - count components with magnitude > 10^-8
    sparse_components = np.sum(np.abs(eigvec) > sparsity_tol)
    sparsity_counts.append(sparse_components)
    sparsity_fraction = sparse_components / total_components
    
    print(f"  Sparsity: {sparse_components}/{total_components} non-zero components")
    print(f"  Sparsity fraction: {sparsity_fraction:.6f}")
    print(f"  Sparsity percentage: {sparsity_fraction * 100:.4f}%")

# Summary statistics
print("\n" + "="*80)
print("SUMMARY:")
print("="*80)

eigenvector_count = len(eiv)
hi_eigenvector_count = sum(is_hi_eigenvector)

print(f"Total eigenvectors analyzed: {eigenvector_count}")
print(f"Eigenvectors that are also Hi eigenvectors: {hi_eigenvector_count}")
print(f"Eigenvectors with Hi eigenvalue ≈ 0: {zero_eigenvalue_count}")

if hi_eigenvector_count > 0:
    valid_hi_eigenvals = [val for val in hi_eigenvalues if val is not None]
    print(f"\nHi eigenvalue statistics:")
    print(f"  Range: [{min(valid_hi_eigenvals):.8f}, {max(valid_hi_eigenvals):.8f}]")
    print(f"  Mean: {np.mean(valid_hi_eigenvals):.8f}")
    print(f"  Unique values: {len(set(np.round(valid_hi_eigenvals, 6)))}")

print(f"\nSparsity statistics:")
print(f"  Min non-zero components: {min(sparsity_counts)}")
print(f"  Max non-zero components: {max(sparsity_counts)}")
print(f"  Mean non-zero components: {np.mean(sparsity_counts):.1f}")
print(f"  Min sparsity fraction: {min(sparsity_counts)/total_components:.6f}")
print(f"  Max sparsity fraction: {max(sparsity_counts)/total_components:.6f}")

# Show distribution of Hi eigenvalues
if hi_eigenvector_count > 0:
    print(f"\nDistribution of Hi eigenvalues:")
    unique_eigenvals, counts = np.unique(np.round([val for val in hi_eigenvalues if val is not None], 6), return_counts=True)
    for eigenval, count in zip(unique_eigenvals, counts):
        is_zero_str = " (≈ 0)" if abs(eigenval) < eigenvalue_tol else ""
        print(f"  {eigenval:.6f}{is_zero_str}: {count} eigenvector(s)")

In [ ]:
# Check if the eigenvectors from Htf are also eigenvectors of H and analyze sparsity
print("Analyzing eigenvectors from Htf (sigma=-18.1) for H eigenvalue properties and sparsity...")

# Tolerance for eigenvalue and sparsity analysis
eigenvalue_tol = 1e-10
sparsity_tol = 1e-8
total_components = 2**20  # 2^20 for 20 qubits

print(f"Total components in each eigenvector: {total_components}")
print(f"Using sparsity threshold: {sparsity_tol}")
print(f"Using eigenvalue consistency threshold: {eigenvalue_tol}")
print("-" * 80)

h_eigenvalues = []
is_h_eigenvector = []
sparsity_counts = []
zero_eigenvalue_count = 0

for i in range(len(eiv)):
    eigvec = eig[:, i]
    htf_eigenval = eiv[i]
    
    print(f"\nEigenvector {i}: Htf eigenvalue = {htf_eigenval:.8f}")
    
    # Test if it's an eigenvector of H
    H_result = H @ eigvec
    
    # Find non-zero components to check eigenvalue consistency
    nonzero_mask = np.abs(eigvec) > eigenvalue_tol
    nonzero_count = np.sum(nonzero_mask)
    
    if nonzero_count > 0:

        # Calculate ratios H*vec / vec for non-zero components
        ratios = H_result[nonzero_mask] / eigvec[nonzero_mask]
        ratio_std = np.std(ratios)
        ratio_mean = np.mean(ratios)
        
        # Check if ratios are consistent (indicating eigenvector property)
        is_eigenvector = ratio_std < eigenvalue_tol        
        
        if is_eigenvector:
            h_eigenval = ratio_mean
            h_eigenvalues.append(hi_eigenval)
            is_h_eigenvector.append(True)
            
            print(f"  ✓ IS eigenvector of H")
            print(f"  H eigenvalue = {h_eigenval:.10f}")
            print(f"  Ratio std deviation = {ratio_std:.2e}")
        else:
            h_eigenvalues.append(None)
            is_h_eigenvector.append(False)
            print(f"  ✗ NOT an eigenvector of H")
            print(f"  Ratio std deviation = {ratio_std:.2e} (too large)")
            print(f"  Sample ratios: {ratios[:5]}")

    else:
        h_eigenvalues.append(None)
        is_h_eigenvector.append(False)
        print(f"  ✗ No significant non-zero components")
    
    # Analyze sparsity - count components with magnitude > 10^-8
    sparse_components = np.sum(np.abs(eigvec) > sparsity_tol)
    sparsity_counts.append(sparse_components)
    sparsity_fraction = sparse_components / total_components
    
    print(f"  Sparsity: {sparse_components}/{total_components} non-zero components")
    print(f"  Sparsity fraction: {sparsity_fraction:.6f}")
    print(f"  Sparsity percentage: {sparsity_fraction * 100:.4f}%")